# Limpeza de Dados do Dataset dos Pilotos

In [75]:
# Import das bibliotecas

import pandas as pd

In [76]:
# Lendo o arquivo csv
df = pd.read_csv('../data/drivers.csv', na_values=["\\N"])
df.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,NaN,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6.0,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14.0,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,NaN,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [77]:
df.dtypes

driverId         int64
driverRef       object
number         float64
code            object
forename        object
surname         object
dob             object
nationality     object
url             object
dtype: object

Há uma inconsistência de tipo de dados na coluna `dob` (que representa a data de aniversário dos pilotos). Essa coluna está definida como `object`, mas representa datas (`datetime64[ns]`).

In [78]:
# Corrigindo o tipo de dados da coluna 'dob' para representar datas
df['dob'] = pd.to_datetime(df['dob'])
df.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,NaN,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6.0,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14.0,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,NaN,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [79]:
df.dtypes

driverId                int64
driverRef              object
number                float64
code                   object
forename               object
surname                object
dob            datetime64[ns]
nationality            object
url                    object
dtype: object

A inconsistência nos tipos das colunas foi corrigida.

## Identificando Dados Ausentes

In [80]:
# Verificando valores ausentes
df.isnull().sum()

driverId         0
driverRef        0
number         802
code           757
forename         0
surname          0
dob              0
nationality      0
url              0
dtype: int64

A coluna `number` possui 802 pilotos do dataset que não possuem número. O campeonato só introduziu o conceito de número de corrida fixo para todos os pilotos a partir de 2014; portanto, os pilotos que encerraram a carreira antes desse ano não possuem esse número.

In [81]:
df[df.code.isnull()]

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
42,43,matta,NaN,NaN,Cristiano,da Matta,1973-09-19,Brazilian,http://en.wikipedia.org/wiki/Cristiano_da_Matta
43,44,panis,NaN,NaN,Olivier,Panis,1966-09-02,French,http://en.wikipedia.org/wiki/Olivier_Panis
44,45,pantano,NaN,NaN,Giorgio,Pantano,1979-02-04,Italian,http://en.wikipedia.org/wiki/Giorgio_Pantano
45,46,bruni,NaN,NaN,Gianmaria,Bruni,1981-05-30,Italian,http://en.wikipedia.org/wiki/Gianmaria_Bruni
46,47,baumgartner,NaN,NaN,Zsolt,Baumgartner,1981-01-01,Hungarian,http://en.wikipedia.org/wiki/Zsolt_Baumgartner
...,...,...,...,...,...,...,...,...,...
802,802,serafini,NaN,NaN,Dorino,Serafini,1909-07-22,Italian,http://en.wikipedia.org/wiki/Dorino_Serafini
803,803,cantrell,NaN,NaN,Bill,Cantrell,1908-01-31,American,http://en.wikipedia.org/wiki/William_Cantrell
804,804,mantz,NaN,NaN,Johnny,Mantz,1918-09-18,American,http://en.wikipedia.org/wiki/Johnny_Mantz
805,805,kladis,NaN,NaN,Danny,Kladis,1917-02-10,American,http://en.wikipedia.org/wiki/Danny_Kladis


O campo da coluna `code` dos 757 pilotos listados acima está nulo, isso indica que eles não possuem um código (abreviação de 3 letras). Essa ausência é esperada, principalmente para pilotos que competiram antes de 2014, quando esses códigos padronizados começaram a ser utilizados.

Para manter o dataset consistente e padronizar um código para cada piloto, foi decidido preencher os valores ausentes da coluna `code` utilizandos códigos artificiais criados a partir dos três primeiros caracteres do nome do piloto na coluna `driverRef`. Porém, esses códigos **NÃO** devem ser tratados como identificadores oficiais da FIA.

In [82]:
# Substituindo os campos vazios na coluna 'code' pelos três primeiros caracteres da coluna 'driverRef' em maiúsculo
df.loc[df.code.isnull(), 'code'] = df['driverRef'].str[:3].str.upper()
df

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44.0,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,NaN,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6.0,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14.0,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,NaN,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen
...,...,...,...,...,...,...,...,...,...
856,858,sargeant,2.0,SAR,Logan,Sargeant,2000-12-31,American,http://en.wikipedia.org/wiki/Logan_Sargeant
857,859,lawson,30.0,LAW,Liam,Lawson,2002-02-11,New Zealander,http://en.wikipedia.org/wiki/Liam_Lawson
858,860,bearman,38.0,BEA,Oliver,Bearman,2005-05-08,British,http://en.wikipedia.org/wiki/Oliver_Bearman
859,861,colapinto,43.0,COL,Franco,Colapinto,2003-05-27,Argentinian,http://en.wikipedia.org/wiki/Franco_Colapinto


## Identificando Dados Duplicados

In [83]:
# Verificação de registros duplicados
df[df.duplicated(keep=False)]

,driverId,driverRef,number,code,forename,surname,dob,nationality,url


In [84]:
# Verificação de registros duplicados apenas na coluna 'driverId'
df[df.duplicated(subset='driverId' ,keep=False)]

,driverId,driverRef,number,code,forename,surname,dob,nationality,url


In [85]:
# Verificação de registros duplicados apenas na coluna 'driverRef'
df[df.duplicated(subset='driverRef' ,keep=False)]

,driverId,driverRef,number,code,forename,surname,dob,nationality,url


Não foi encontrado nenhum registro duplicado.

## Análise da coluna `nationality`

In [86]:
df['nationality'].value_counts()

British              166
American             158
Italian               99
French                73
German                50
Brazilian             32
Argentine             24
Belgian               23
Swiss                 23
South African         23
Japanese              20
Australian            19
Dutch                 18
Spanish               15
Austrian              15
Canadian              14
Swedish               10
New Zealander         10
Finnish                9
Mexican                6
Danish                 5
Irish                  5
Rhodesian              4
Portuguese             4
Russian                4
Monegasque             4
Uruguayan              4
Colombian              3
Venezuelan             3
East German            3
Thai                   2
Indian                 2
Indonesian             1
Chinese                1
Argentine-Italian      1
Czech                  1
American-Italian       1
Liechtensteiner        1
Chilean                1
Malaysian              1


A análise da coluna `nationality` revelou uma inconsistência nos padrões dos dados: 24 registros com o valor `Argentine` e 1 registro com o valor `Argentinian`, embora ambos representem a mesma nacionalidade.

In [87]:
argentinian = df[(df.forename == 'Franco') & (df.surname == 'Colapinto')]
argentinian['nationality'].values[0]

'Argentinian '

Além da inconsistência mencionada anteriormente, ao buscar pelo piloto de nacionalidade `Argentinian`, foi identificado que o valor presente nesse campo contém um espaço extra, dificultando a sua busca. Dessa forma, a remoção de espaços extras será efetuada em todos os valores na coluna `nationality`, de modo a garantir a padronização e evitando que a inconsistência ocorra em outros registros.

In [88]:
# Removendo espaços extras nos valores da coluna 'nationality'
df['nationality'] = df['nationality'].str.strip()
df[(df.nationality == 'Argentine') | (df.nationality == 'Argentinian')]

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
59,60,mazzacane,NaN,MAZ,Gastón,Mazzacane,1975-05-08,Argentine,http://en.wikipedia.org/wiki/Gast%C3%B3n_Mazza...
73,74,tuero,NaN,TUE,Esteban,Tuero,1978-04-22,Argentine,http://en.wikipedia.org/wiki/Esteban_Tuero
81,82,fontana,NaN,FON,Norberto,Fontana,1975-01-20,Argentine,http://en.wikipedia.org/wiki/Norberto_Fontana
164,165,larrauri,NaN,LAR,Oscar,Larrauri,1954-08-19,Argentine,http://en.wikipedia.org/wiki/Oscar_Larrauri
198,199,reutemann,NaN,REU,Carlos,Reutemann,1942-04-12,Argentine,http://en.wikipedia.org/wiki/Carlos_Reutemann
214,215,guerra,NaN,GUE,Miguel Ángel,Guerra,1953-08-31,Argentine,http://en.wikipedia.org/wiki/Miguel_Angel_Guerra
216,217,zunino,NaN,ZUN,Ricardo,Zunino,1949-04-13,Argentine,http://en.wikipedia.org/wiki/Ricardo_Zunino
438,439,estefano,NaN,EST,Nasif,Estéfano,1932-11-18,Argentine,http://en.wikipedia.org/wiki/Nasif_Est%C3%A9fano
485,485,bordeu,NaN,BOR,Juan Manuel,Bordeu,1934-01-28,Argentine,http://en.wikipedia.org/wiki/Juan_Manuel_Bordeu
496,496,menditeguy,NaN,MEN,Carlos,Menditeguy,1914-08-10,Argentine,http://en.wikipedia.org/wiki/Carlos_Menditeguy


A partir disso, a nacionalidade do piloto Franco Colapinto será modificada para `Argentine` para manter a consistência dos dados.

In [89]:
# Alterando o valor 'Argentinian' para 'Argentine'
df.loc[df.nationality == 'Argentinian', 'nationality'] = 'Argentine'
df[(df.forename == 'Franco') & (df.surname == 'Colapinto')]

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
859,861,colapinto,43.0,COL,Franco,Colapinto,2003-05-27,Argentine,http://en.wikipedia.org/wiki/Franco_Colapinto


## Salvando o Dataset Tratado

In [90]:
df.to_csv('../data_cleaned/drivers.csv', index=False)